<a href="https://colab.research.google.com/github/qriovider/queueing_theory/blob/main/preemptive_2type_jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpy

In [2]:
import simpy
import numpy as np
import matplotlib.pyplot as plt

In [28]:
LAMBDA_1 = 2
LAMBDA_2 = 1
MU_1 = 4
MU_2 = 3

def generate_interarrival(LAMBDA):
  return np.random.exponential(1./LAMBDA)

def generate_service(MU):
  return np.random.exponential(1./MU)

def arriving_1(env, resource):
  i = 0
  while True:
    i = i + 1
    job1 = Job1(name="Job1-{}".format(i))
    yield env.timeout(generate_interarrival(LAMBDA_1))
    print(env.now, job1.name, 'arrives')
    env.process(job1.resource_user_1(env, i, resource))

def arriving_2(env, resource):
  i = 0
  while True:
    i = i + 1
    job2 = Job2(name="Job2-{}".format(i))
    yield env.timeout(generate_interarrival(LAMBDA_2))
    print(env.now, job2.name, 'arrives')
    env.process(job2.resource_user_2(env, i, resource))


class Job1(object):
  def __init__(self, name):
    self.name = name

  def resource_user_1(self, env, i, resource):
    with resource.request(priority=1) as req:
      yield req
      print(env.now, self.name, 'is being served')
      print('The number of using resources is', resource.count)
      yield env.timeout(generate_service(MU_1))
      print(env.now, self.name, 'leaves')

class Job2(object):
  def __init__(self, name):
    self.name = name
  
  def resource_user_2(self, env, i, resource):
    with resource.request(priority=2) as req:
      try:
        yield req
        print('The number of using resources is', resource.count)
        print(env.now, self.name, 'is being served')
        yield env.timeout(generate_service(MU_2))
        print(env.now, self.name, 'leaves')
      except simpy.Interrupt as interrupt:  
        print(env.now, self.name, 'got preempted')


In [30]:
env = simpy.Environment()
resource = simpy.PreemptiveResource(env, capacity=2)
env.process(arriving_1(env, resource))
env.process(arriving_2(env, resource))
env.run(until=50)

0.006025114949002309 Job1-1 arrives
0.006025114949002309 Job1-1 is being served
The number of using resources is 1
0.03569920720807038 Job1-1 leaves
0.17851745808065358 Job1-2 arrives
0.17851745808065358 Job1-2 is being served
The number of using resources is 1
0.21170394101904771 Job1-2 leaves
0.6442622454438245 Job2-1 arrives
The number of using resources is 1
0.6442622454438245 Job2-1 is being served
0.9246984154955897 Job2-2 arrives
The number of using resources is 2
0.9246984154955897 Job2-2 is being served
1.1210314729551734 Job2-2 leaves
1.6768808036849658 Job1-3 arrives
1.6768808036849658 Job1-3 is being served
The number of using resources is 2
1.7493896889758371 Job2-1 leaves
2.0087200249898918 Job1-3 leaves
2.226446134942936 Job1-4 arrives
2.226446134942936 Job1-4 is being served
The number of using resources is 1
2.3517283955348627 Job1-5 arrives
2.3517283955348627 Job1-5 is being served
The number of using resources is 2
2.5887446003829693 Job1-4 leaves
2.625589119387119 J